In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

from d2l.tensorflow import config, data, plot

config.setup('CPU')

Tensorflow running on CPU


In [2]:
def vgg_block(num_convs, num_channels):
    blk = keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(keras.layers.Conv2D(filters=num_channels, kernel_size=3, padding='same', activation='relu'))
    blk.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

In [3]:
def vgg(conv_arch):
    net = keras.models.Sequential()
    
    for num_convs, num_channels in conv_arch:
        net.add(vgg_block(num_convs, num_channels))

    net.add(keras.layers.Flatten())
    
    net.add(keras.layers.Dense(units=4096, activation='relu'))
    net.add(keras.layers.Dropout(rate=0.5))
    net.add(keras.layers.Dense(units=4096, activation='relu'))
    net.add(keras.layers.Dropout(rate=0.5))
    
    net.add(keras.layers.Dense(units=10, activation='softmax'))

    net.build((None, 224, 224, 1))
    return net

In [4]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 256), (2, 512), (2, 512))

vgg11 = vgg(conv_arch)

In [5]:
vgg11.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    multiple                  640       
_________________________________________________________________
sequential_2 (Sequential)    multiple                  73856     
_________________________________________________________________
sequential_3 (Sequential)    multiple                  885248    
_________________________________________________________________
sequential_4 (Sequential)    multiple                  1180160   
_________________________________________________________________
sequential_5 (Sequential)    multiple                  3539968   
_________________________________________________________________
sequential_6 (Sequential)    multiple                  4719616   
_________________________________________________________________
flatten (Flatten)            multiple                  0

In [6]:
X = np.random.uniform(size=(1, 224, 224, 1)).astype(np.float32)
np.random.uniform()

for layer in vgg11.layers:
    X = layer(X)
    print('{} output shape: {}'.format(layer.name, X.shape))

sequential_1 output shape: (1, 112, 112, 64)
sequential_2 output shape: (1, 56, 56, 128)
sequential_3 output shape: (1, 28, 28, 256)
sequential_4 output shape: (1, 14, 14, 256)
sequential_5 output shape: (1, 7, 7, 512)
sequential_6 output shape: (1, 3, 3, 512)
flatten output shape: (1, 4608)
dense output shape: (1, 4096)
dropout output shape: (1, 4096)
dense_1 output shape: (1, 4096)
dropout_1 output shape: (1, 4096)
dense_2 output shape: (1, 10)


In [8]:
ratio = 8
small_conv_arch = [(c, f // ratio) for c, f in conv_arch]
small_vgg11 = vgg(small_conv_arch)
small_vgg11.build(input_shape=(None, 224, 224, 1))

In [9]:
batch_size = 128
epochs = 5
learning_rate = 0.03

optmizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

small_vgg11.compile(optimizer=optmizer, loss=loss, metrics=metric)

In [10]:
X_train, y_train, X_test, y_test = data.load_fashion_mnist_keras()

In [11]:
train_iter = data.resize_images_generator(X_train, y_train, batch_size, [224, 224])
test_iter = data.resize_images_generator(X_test, y_test, batch_size, [224, 224])

train_steps = len(X_train) // batch_size + 1
val_steps = len(X_test) // batch_size + 1

In [12]:
# hist = small_vgg11.fit(train_iter, epochs=epochs, validation_data=test_iter, steps_per_epoch=train_steps, validation_steps=val_steps)

In [13]:
# plot.plot_history(hist, 'categorical_accuracy')